# MCPAdapt - AI Tinkerer Lausanne - 16.06.2025

<p align="center">
  <img src="https://grll.github.io/mcpadapt/assets/logo_dark_transparent_cropped.png" alt="MCPAdapt Logo" width="200">
</p>


![PyPI version](https://img.shields.io/pypi/v/mcpadapt)
![Python versions](https://img.shields.io/pypi/pyversions/mcpadapt)
![Tests](https://github.com/grll/mcpadapt/actions/workflows/ci.yml/badge.svg)
![License](https://img.shields.io/github/license/grll/mcpadapt)
![PyPI downloads](https://img.shields.io/pypi/dm/mcpadapt)
![GitHub Repo stars](https://img.shields.io/github/stars/grll/mcpadapt)

For more context, guides and API references visit our new [documentation](https://grll.github.io/mcpadapt/).

Unlock 650+ MCP servers tools in your favorite agentic framework.

Model Context Protocol is an open-source protocol introduced by Anthropic which allows anyone to simply and quickly make tools and resources available as "MCP Servers".

Since its release more than 650 MCP servers have been created giving access to many data & tools to supported "MCP Client".

This project makes calling any MCP servers tools seemless from any agentic framework. Virtually providing your agentic worfklow access to 650+ MCP servers tools.

Look at [glama.ai](https://glama.ai/mcp/servers) or [smithery.ai](https://smithery.ai/) to give you an idea of what your agent could access.

## What is MCPAdapt?

<p align="center">
    <img src="./images/mcpadapt_excalidraw.png" alt="mcpadapt diagram"  width="800">
</p>

MCPAdapt help you create adapters between agentic frameworks and MCP Servers.

## A running example

### Installation

In [3]:
!uv add mcpadapt[smolagents,crewai]

Resolved 222 packages in 13ms
Audited 155 packages in 0.63ms


### PubMed MCP with Smolagents

In [4]:
import os

from mcp import StdioServerParameters
from smolagents import CodeAgent, HfApiModel  # type: ignore

from mcpadapt.core import MCPAdapt
from mcpadapt.smolagents_adapter import SmolAgentsAdapter

with MCPAdapt(
    StdioServerParameters(
        command="uvx",
        args=["--quiet", "pubmedmcp@0.1.3"],
        env={"UV_PYTHON": "3.12", **os.environ},
    ),
    SmolAgentsAdapter(),
) as tools:
    print(tools[0](request={"term": "efficient treatment hangover"}))
    # agent = CodeAgent(tools=tools, model=HfApiModel())
    # agent.run("Find studies about hangover?")

1. Plants (Basel). 2025 May 31;14(11):1692. doi: 10.3390/plants14111692.

In Vitro Conditions Research of Sophora koreensis Nakai for Shoot Elongation.

Lee H(1), Han GI(2), Cheon KS(1), Cheong EJ(3).

Author information:
(1)Division of Tree Improvement and Biotechnology, Department of Forest 
Bio-Resources, National Institute of Forest Science, Suwon 16631, Republic of 
Korea.
(2)Division of Forest Conservation and Restoration, Baekdudaegan National 
Arboretum, Bonghwa 36209, Republic of Korea.
(3)Division of Forest Science, College of Forest and Environmental Sciences, 
Kangwon National University, Chuncheon 24341, Republic of Korea.

Sophora koreensis Nakai, listed as endangered on the IUCN Red List, is a species 
native to Korea, specifically found in parts of Gangwon-do. Recent research 
highlights its potential in hangover relief and as an antioxidant source, 
sparking interest in enhancing its components through mutation for commercial 
purposes. Given its limited distribution, 

### PubMed MCP with CrewAI

In [ ]:
import os

from crewai import Agent, Crew, Task  # type: ignore
from dotenv import load_dotenv
from mcp import StdioServerParameters

from mcpadapt.core import MCPAdapt
from mcpadapt.crewai_adapter import CrewAIAdapter

load_dotenv()
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY is not set. Create a .env file at the root of the project with OPENAI_API_KEY=<your-api-key>"
    )

with MCPAdapt(
    StdioServerParameters(
        command="uvx",
        args=["--quiet", "pubmedmcp@0.1.3"],
        env={"UV_PYTHON": "3.12", **os.environ},
    ),
    CrewAIAdapter(),
) as tools:
    print(tools[0].run(request={"term": "efficient treatment hangover"}))
    print(tools[0])
    # print(tools[0].description)
    # Create a simple agent with the pubmcp tool
    agent = Agent(
        role="Research Agent",
        goal="Find studies about hangover",
        backstory="You help find studies about hangover",
        verbose=True,
        tools=[tools[0]],
    )

    # Create a task
    task = Task(
        description="Find studies about hangover",
        agent=agent,
        expected_output="A list of studies about hangover",
    )

    # Create a crew
    crew = Crew(agents=[agent], tasks=[task], verbose=True)

    # Run the crew
    crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5651b6e6-3d4e-4f9d-8c0f-39d5bf397e6c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Research Agent

## Task: Find studies about hangover

Output()

## Taking a step back: Why do we need Adapter?

Comparing the various Tool Class implementation

### Somlagents Tool Class

```python
class Tool:
    def __init__(self, *args, **kwargs):
        # Initialize tool with is_initialized flag
        ...
    
    def validate_arguments(self):
        # Validate required attributes and tool configuration
        ...
    
    def forward(self, *args, **kwargs):
        # Main method to implement tool logic (must be overridden)
        ...
    
    def __call__(self, *args, sanitize_inputs_outputs: bool = False, **kwargs):
        # Execute tool with optional input/output sanitization
        ...
    
    def setup(self):
        # Perform expensive operations before first use
        ...
    
    def to_dict(self) -> dict:
        # Convert tool to dictionary representation
        ...
    
    @classmethod
    def from_dict(cls, tool_dict: dict[str, Any], **kwargs) -> "Tool":
        # Create tool from dictionary representation
        ...
    
    def save(self, output_dir: str | Path, tool_file_name: str = "tool", make_gradio_app: bool = True):
        # Save tool code and optionally create Gradio UI
        ...
    
    def push_to_hub(self, repo_id: str, commit_message: str = "Upload tool", private: bool | None = None, token: bool | str | None = None, create_pr: bool = False) -> str:
        # Upload tool to Hugging Face Hub
        ...
    
    @classmethod
    def from_hub(cls, repo_id: str, token: str | None = None, trust_remote_code: bool = False, **kwargs):
        # Load tool from Hugging Face Hub
        ...
    
    @classmethod
    def from_code(cls, tool_code: str, **kwargs):
        # Create tool from source code string
        ...
    
    @staticmethod
    def from_space(space_id: str, name: str, description: str, api_name: str | None = None, token: str | None = None):
        # Create tool from Hugging Face Space
        ...
    
    @staticmethod
    def from_gradio(gradio_tool):
        # Create tool from Gradio tool
        ...
    
    @staticmethod
    def from_langchain(langchain_tool):
        # Create tool from LangChain tool
        ...
```

### CrewAI Tool Class

```python
class BaseTool(BaseModel, ABC):
    def __init__(self, **kwargs):
        # Initialize tool with name, description, args_schema, and usage tracking
        ...
    
    def run(self, *args: Any, **kwargs: Any) -> Any:
        # Execute tool, track usage, handle async if needed
        ...
    
    def reset_usage_count(self) -> None:
        # Reset the current usage count to zero
        ...
    
    @abstractmethod
    def _run(self, *args: Any, **kwargs: Any) -> Any:
        # Actual implementation of the tool (must be overridden)
        ...
    
    def to_structured_tool(self) -> CrewStructuredTool:
        # Convert to CrewStructuredTool instance
        ...
    
    @classmethod
    def from_langchain(cls, tool: Any) -> "BaseTool":
        # Create Tool instance from LangChain tool
        ...
    
    def _set_args_schema(self):
        # Set argument schema from function annotations
        ...
    
    def _generate_description(self):
        # Generate detailed description with argument info
        ...
    
    @staticmethod
    def _get_arg_annotations(annotation: type[Any] | None) -> str:
        # Get string representation of type annotations
        ...


class Tool(BaseTool):
    def _run(self, *args: Any, **kwargs: Any) -> Any:
        # Execute the func callable
        ...
    
    @classmethod
    def from_langchain(cls, tool: Any) -> "Tool":
        # Create Tool instance from LangChain tool with func attribute
        ...
```

### Python MCP Client SDK

While MCP Client input / output take a gain a different shape and is async...

```python
async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read, write, sampling_callback=handle_sampling_message
        ) as session:
            # Initialize the connection
            await session.initialize()

            # List available prompts
            prompts = await session.list_prompts()

            # Get a prompt
            prompt = await session.get_prompt(
                "example-prompt", arguments={"arg1": "value"}
            )

            # List available resources
            resources = await session.list_resources()

            # List available tools
            tools = await session.list_tools()

            # Read a resource
            content, mime_type = await session.read_resource("file://some/path")

            # Call a tool
            result = await session.call_tool("tool-name", arguments={"arg1": "value"})
```

* Each agent will need some custom code to integrate it into their framework and wrap it into their tools.
* We can make it easy to write such wrappers = MCPAdapt
* Some features are very useful all across (async to sync, json schema to pydantic, etc...) 

## Creating an Adapter

```python
class YourFrameworkAdapter(ToolAdapter):
    def adapt(
        self,
        func: Callable[[dict | None], mcp.types.CallToolResult],
        mcp_tool: mcp.types.Tool,
    ) -> YourFramework.Tool:
        # HERE implement how the adapter should convert a simple function and mcp_tool (JSON Schema)
        # into your framework tool. see smolagents_adapter.py for an example
    
    def async_adapt(
        self,
        afunc: Callable[[dict | None], Coroutine[Any, Any, mcp.types.CallToolResult]],
        mcp_tool: mcp.types.Tool,
    ) -> YourFramework.Tool:
        # if your framework supports async function even better use async_adapt.
```

```python
class Tool(BaseModel):
    """Definition for a tool the client can call."""

    name: str
    """The name of the tool."""
    description: str | None = None
    """A human-readable description of the tool."""
    inputSchema: dict[str, Any]
    """A JSON Schema object defining the expected parameters for the tool."""
    annotations: ToolAnnotations | None = None
    """Optional additional tool information."""
    model_config = ConfigDict(extra="allow")
```

## Some Considerations

### MCP Python SDK is async, Smolagents is sync

```python
    def _run_loop(self):
        """Runs the event loop in a separate thread (for synchronous usage)."""
        asyncio.set_event_loop(self.loop)

        async def setup():
            async with AsyncExitStack() as stack:
                connections = [
                    await stack.enter_async_context(
                        mcptools(params, self.client_session_timeout_seconds)
                    )
                    for params in self.serverparams
                ]
                self.sessions, self.mcp_tools = [list(c) for c in zip(*connections)]
                self.ready.set()  # Signal initialization is complete
                await asyncio.Event().wait()  # Keep session alive until stopped

        self.task = self.loop.create_task(setup())
        try:
            self.loop.run_until_complete(self.task)
        except asyncio.CancelledError:
            pass
```

### Support for more MCP features: Resources

* tools are not ambigious
* resources in MCP specs are not fully clear in the context of Agents
    * Claude Desktop shows a menu the user can select from with resources to add them to context
    * Agent should it decides by itself or let the end user decide?

## Future of MCPAdapt

* integrate with more into agentic frameworks
* becoming more of a Client for Agentic framework dealing with dilema above
* support for more than just tools: resources, prompts, and so on


Give us a star!

<p align="center">
    <img src="./images/frame.png">
</p>